# KNN

In [11]:
from collections import Counter
import numpy as np

def euclidean_distance(x1, x2):
    return np.sqrt(np.sum((x1 - x2)**2))

In [12]:
import numpy as np
from scipy.stats import mode

class KNN:
    def __init__(self, k=3):
        self.k = k

    def fit(self, x, y):
        self.x = x
        self.y = y

    def predict(self, x_test):
        return [self._predict(x) for x in x_test]

    def _predict(self, x):
        distances = self._compute_distances(x)
        nearest_indices = self._find_k_nearest(distances)
        return self._classify(nearest_indices)

    def _compute_distances(self, x):
        return np.sqrt(np.sum((self.x - x) ** 2, axis=1))

    def _find_k_nearest(self, distances):
        return np.argsort(distances)[:self.k]

    def _classify(self, nearest_indices):
        nearest_labels = self.y[nearest_indices]
        return mode(nearest_labels)[0][0]


# Naive Bayes

In [13]:
class NaiveBayes:
    def fit(self, X, y):
        self._classes = np.unique(y)
        n_classes = len(self._classes)

        # calculate mean, var, and prior for each class
        self._mean = np.zeros((n_classes, X.shape[1]), dtype=np.float64)
        self._var = np.zeros((n_classes, X.shape[1]), dtype=np.float64)
        self._priors =  np.zeros(n_classes, dtype=np.float64)

        for idx, c in enumerate(self._classes):
            X_c = X[y==c]
            self._mean[idx, :] = X_c.mean(axis=0)
            self._var[idx, :] = X_c.var(axis=0)
            self._priors[idx] = X_c.shape[0] / float(X.shape[0])

    def predict(self, X):
        return [self._predict(x) for x in X]

    def _predict(self, x):
        posteriors = []

        # calculate posterior probability for each class
        for idx, c in enumerate(self._classes):
            prior = np.log(self._priors[idx])
            posterior = np.sum(np.log(self._pdf(idx, x)))
            posterior = prior + posterior
            posteriors.append(posterior)

        # return class with highest posterior probability
        return self._classes[np.argmax(posteriors)]
        
    def _pdf(self, class_idx, x):
        mean = self._mean[class_idx]
        var = self._var[class_idx]
        numerator = np.exp(-((x - mean) ** 2) / (2 * var))
        denominator = np.sqrt(2 * np.pi * var)
        return numerator / denominator
        

In [14]:

if __name__ == "__main__":
    # Imports
    from matplotlib.colors import ListedColormap
    from sklearn import datasets
    from sklearn.model_selection import train_test_split

    def accuracy(y_true, y_pred):
        accuracy = np.sum(y_true == y_pred) / len(y_true)
        return accuracy

    iris = datasets.load_iris()
    X, y = iris.data, iris.target

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=1234
    )

    k = 3
    clf = KNN(k=k)
    clf.fit(X_train, y_train)
    predictions = clf.predict(X_test)
    print("KNN classification accuracy", accuracy(y_test, predictions))

    # implementing from library

    from sklearn.neighbors import KNeighborsClassifier
    knn = KNeighborsClassifier(n_neighbors=3)
    knn.fit(X_train, y_train)
    pred = knn.predict(X_test)
    print('Knn classification accuracy (library)', accuracy(y_test, pred))

    nb = NaiveBayes()
    nb.fit(X_train, y_train)
    predictions = nb.predict(X_test)
    print("Naive Bayes classification accuracy", accuracy(y_test, predictions))

    # implementing from library
    from sklearn.naive_bayes import GaussianNB
    gnb = GaussianNB()
    model = gnb.fit(X_train, y_train)
    preds = gnb.predict(X_test)

    # Evaluate accuracy
    print("Naive Bayes classification accuracy (library)", accuracy(y_test, preds))

KNN classification accuracy 1.0
Knn classification accuracy (library) 1.0
Naive Bayes classification accuracy 1.0
Naive Bayes classification accuracy (library) 1.0


/var/folders/5f/1b1zms0j03d6wzq402h8fw8c0000gn/T/ipykernel_42447/4009852673.py:28: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  return mode(nearest_labels)[0][0]
